# Comparing the forecast performance of a process-based and model-free forecasting model


Empirical Dynamical Modelling is a modeling technique that relies on Takens theorem. This says, that we can reconstruct a representation of the system dynamics from system state measurements, available as an univariate timeseries (https://sugiharalab.github.io/EDM_Documentation/edm_intro/). When we want to make forecasts about a system, this technique can be applied in cases where no process model is available.

Are mechanistic models that condense theory of our systems still useful? How does the EDM approach perform in forecasting in comparison to a (good) process model? This is the question we will try to answer in a pure simulation framework. We use an EDM and a process model $M_{f}$ to forecast the dynamics that we itself generate with an observation model $M_{obs}$.

The goodness of the forecast model will be varied through the parameter for the intrinsic growth rate, $r$ (= *lambda* in models.py). Use as forecast model $M_{f}$ models.Ricker_Single (without temperature effects) and as observation model $M_{obs}$ models.Ricker_Multi (see models.py).
1. Setting: We assume we have good knowledge about $r$ in the process model. Use the same $r$ for the process model and the observation model and vary simultaneously.
2. Setting: We assume stepwise decreasing knowledge about the true parameter $r$, by increasing difference in $r_{obs}$ and $r_{f}$. Fix $r_{f}$ and slowly increase $r_{obs}$.
3. Compare the $M_{f}$ Ricker and the EDM in "goodness" (using for example $R^2$) across $r$ for both settings.
4. Use a gradient of goodness (referring to a gradient in $r$) to assess the relationship between maximum and real forecast horizon.

Final hypothesis:
$\Delta(h_{real},h_{max}) \sim g(f(y), y) + WPE (y)$
Where $g$ is some function of model goodness and WPE the weighted permutation entropy.


### The forecast horizon

The forecast horizon is the limit in time, after which the proficiency (some distance, accuracy or association measure $d$) of the ensemble forecast to forecast the observation on average falls below a forecast proficiency threshold $\rho$.
Formalized, the forecast horizon $h$ is defined as
$h := min_{t}(E[d(y_{f,i}, y_{obs})] < \rho)$.
We compute the forecast horizon of the model twice: Once for the observations $y_{obs}$ and once for the model itself, the mean ensemble trajectory $overline{Y_{f}}$. While the latter we call the real forecast horizon, $h_{real}$, the former will be termed the maximum forecast horizon $h_{max}$
$h_{max} := min_{t}(E[overline{Y_{f}}, y_{obs})] < \rho)$.

### The weighted permutation entropy

The (weighted) permutation entropy can be taken as a measure of intrinsic predictability of a system, given measurements of its state variable. It's robust to observational and process noise even if the dynamic exhibits chaotic behaviour. In the study of Pennenkamp et al. 2019, the relationship between the forecast error and the WPE was found to be positive. As such, we can expect the WPE to have a negative effect also on the difference between the maximum and the real forecast horizon by affecting both, the realisable forecast horizon and the model goodness, quantified as $g$.



# Code examples

Load the required packages.

In [10]:
import sys
sys.path.append('../Ricker')

import models
from scipy.stats import pearsonr, ttest_ind, rankdata
from sklearn.metrics import r2_score
import numpy as np
from collections import Counter
from math import factorial


### Compute the real and the maximum forecast horizon of the model.

2. Simulate observations and an according forecast ensemble under the specific choices of parameters for both, the observation model and the forecast model.
3. Compute the model goodness, here as $R^2$
4. Compute the forecast horizon based on a proficiency metric and a forecast proficiency threshold (here: Pearson's r with a threshold of 0.5).
5. Determine the difference $\Delta$ in $h_{max}$ and $h_{min}$ with a t-test.

In [11]:
def simulate():

    init_s = 0.99
    r = 0.05
    r1= 0.05
    r2= 0.05

    mf_pars = {'lambda': r, 'K': 1}
    mf_errs = {"sigma":0.0001,"phi":0.0, "init_u":1e-4}

    mo_pars = {'lambda1': r1+0.0001, 'K1': 1, 'alpha':1, 'beta':0.00006, 'lambda2': r2, 'K2': 1, 'gamma':1, 'delta':0.00005}
    mo_errs = {"sigma":0.0001,"phi":0.0002, "init_u":1e-4}

    sf_pars = {"iterations":2*52, "initial_size": init_s, "ensemble_size": 25}
    so_pars = {"iterations":2*52, "initial_size": (init_s, init_s), "ensemble_size": 1}

    mf = models.Ricker_Single(set_seed=False)
    mo = models.Ricker_Multi(set_seed=False)

    mf.parameters(mf_pars, mf_errs)
    mo.parameters(mo_pars, mo_errs)

    yf = mf.simulate(sf_pars)['ts']
    yo = mo.simulate(so_pars)['ts'][:,:,0].squeeze()

    return yf, yo

def proficiency(yf, yo):

    window = 3

    d = []
    for j in range(len(yo) - window):
        d.append(pearsonr(yo[j:j + window], yf[j:j + window])[0])

    return np.array(d)

def model_goodness(yf, yo):

    r2 = []
    for i in range(yf.shape[0]):
        r2.append(r2_score(yo, yf[i, :]))
    return np.array(r2)

def horizon(yf, yo, rho = 0.5):

    p = []
    for i in range(yf.shape[0]):

        p.append(proficiency(yf[i,:], yo))

    p = np.array(p)
    p_reached = (np.mean(p, axis=0) < rho)

    if p_reached.sum() == 0:
        fh = len(yo)
    else:
        fh = np.argmax(p_reached)

    return fh

fh_max = []
fh_real = []
for i in range(20):
    yf, yo = simulate()
    r2 = model_goodness(yf, yo)
    fh_max.append(horizon(yf, yf.mean(axis=0)))
    fh_real.append(horizon(yf, yo))

np.round(ttest_ind(fh_max, fh_real)[0], 4)



19.0035

### Example computation of the permutation entropy.

In [12]:
# based on Pennekamp 2019
# PE based on an embedded time series
def embed(x, m, d = 1):
    """
    Pennekamp 2019
    """
    n = len(x) - (m-1)*d
    X = np.arange(len(x))
    out = np.array([X[np.arange(n)]]*m)
    a = np.repeat(np.arange(1, m)*d, out.shape[1])
    out[1:,] = out[1:,]+a.reshape(out[1:,].shape)
    out = x[out]

    return out

def entropy(wd):
    """
    in bits
    """
    return -np.sum(list(wd.values())*np.log2(list(wd.values())))


def word_distr(x_emb, tie_method='average'):

    words = [np.array2string(rankdata(x_emb[:, i])) for i in range(x_emb.shape[1])]
    c = dict(Counter(words))
    for k, v in c.items():
        c[k] = v/len(words)
    return c

def permutation_entropy(x, m, d=1):

    x_emb = embed(x, m=m, d=d)
    wd = word_distr(x_emb)
    denom = np.log2(2 * factorial(m))
    ent = entropy(wd) / denom

    return ent


x = np.random.normal(0,1,30)
permutation_entropy(x, m = 3)



0.6678522378330369